# Examples

Some examples of how to use the anomaly agent.

<a target="_blank" href="https://colab.research.google.com/github/andrewm4894/anomaly-agent/blob/main/notebooks/examples.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [1]:
import os
from anomaly_agent.utils import make_df, make_anomaly_config
from anomaly_agent.plot import plot_df
from anomaly_agent.agent import AnomalyAgent

# set openai api key if not in environment
# os.environ['OPENAI_API_KEY'] = "<your-openai-api-key>"

In [2]:
# get and anomaly config to generate some dummy data
anomaly_cfg = make_anomaly_config()
print(anomaly_cfg)

# generate some dummy data
df = make_df(100, 3, anomaly_config=anomaly_cfg)
df.head()


{'enabled': True, 'fraction': 0.1, 'methods': ['spike', 'drop', 'shift', 'noise'], 'spike_factor': 10, 'shift_value': 3, 'noise_std': 0.2}


,timestamp,var1,var2,var3
0,2020-01-01,0.057295,0.045153,0.474300
1,2020-01-02,0.876011,0.080676,0.135618
2,2020-01-03,0.586661,0.535590,0.270697
3,2020-01-04,0.514778,0.864993,0.561710
4,2020-01-05,0.385586,0.351691,3.051536


In [3]:
# plot the data
plot_df(df)

In [ ]:
# create anomaly agent
anomaly_agent = AnomalyAgent()

# detect anomalies
anomalies = anomaly_agent.detect_anomalies(df)

# detect anomalies without verification
anomalies_no_verify = anomaly_agent.detect_anomalies(df, verify=False)

# print anomalies
print(anomalies)


{'var1': AnomalyList(anomalies=[Anomaly(timestamp='2020-01-13', variable_value=8.413354, anomaly_description='Significant outlier, value much higher than surrounding data.'), Anomaly(timestamp='2020-01-31', variable_value=8.007763, anomaly_description='Significant outlier, value much higher than surrounding data.'), Anomaly(timestamp='2020-03-03', variable_value=2.405734, anomaly_description='Notable spike compared to previous values.'), Anomaly(timestamp='2020-03-09', variable_value=2.336758, anomaly_description='Notable spike compared to previous values.'), Anomaly(timestamp='2020-03-30', variable_value=3.348009, anomaly_description='Notable spike compared to previous values.')]), 'var2': AnomalyList(anomalies=[Anomaly(timestamp='2020-01-14', variable_value=2.765644, anomaly_description='Significant spike in value, greatly exceeds previous values.'), Anomaly(timestamp='2020-02-08', variable_value=3.644783, anomaly_description='Sudden spike, value is unusually high compared to surroun

In [5]:
# get anomalies in long format
df_anomalies_long = anomaly_agent.get_anomalies_df(anomalies)
df_anomalies_long.head()

,timestamp,variable_name,value,description
0,2020-01-13,var1,8.413354,"Significant outlier, value much higher than su..."
1,2020-01-31,var1,8.007763,"Significant outlier, value much higher than su..."
2,2020-03-03,var1,2.405734,Notable spike compared to previous values.
3,2020-03-09,var1,2.336758,Notable spike compared to previous values.
4,2020-03-30,var1,3.348009,Notable spike compared to previous values.


In [6]:
# get anomalies in wide format
df_anomalies_wide = anomaly_agent.get_anomalies_df(anomalies, format="wide")

df_anomalies_wide_no_verify = anomaly_agent.get_anomalies_df(anomalies_no_verify, format="wide")

df_anomalies_wide.head()


,timestamp,var1,var3,var2
0,2020-01-13,8.413354,NaN,NaN
1,2020-01-31,8.007763,NaN,NaN
2,2020-03-03,2.405734,3.761072,NaN
3,2020-03-09,2.336758,NaN,NaN
4,2020-03-30,3.348009,NaN,NaN


In [7]:
# merge anomalies with original data
df_final = df.merge(df_anomalies_wide, on="timestamp", how="left", suffixes=("", "_anomaly_flag"))

df_final_no_verify = df.merge(df_anomalies_wide_no_verify, on="timestamp", how="left", suffixes=("", "_anomaly_flag"))

df_final.head()


,timestamp,var1,var2,var3,var1_anomaly_flag,var3_anomaly_flag,var2_anomaly_flag
0,2020-01-01,0.057295,0.045153,0.474300,NaN,NaN,NaN
1,2020-01-02,0.876011,0.080676,0.135618,NaN,NaN,NaN
2,2020-01-03,0.586661,0.535590,0.270697,NaN,NaN,NaN
3,2020-01-04,0.514778,0.864993,0.561710,NaN,NaN,NaN
4,2020-01-05,0.385586,0.351691,3.051536,NaN,3.051536,NaN


In [8]:
# plot final data with anomalies
fig = plot_df(df_final, return_fig=True)
fig.show()

In [9]:
# plot final data no verify
fig = plot_df(df_final_no_verify, return_fig=True)
fig.show()